# Note: combined_df_v2.csv is our initial dataset containing high-level planning and low-level reasoning steps

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

path = "combined_df_v2.csv"

df = pd.read_csv(path)

df.head()





,custom_id,question,answer,LLR,HLR
0,req_330000,Will's breakfast supplied him 900 calories of ...,Step 1: Calculate the duration of jogging in m...,[('Step 1: Calculate the duration of jogging i...,['Step 1: Calculate the duration of jogging in...
1,req_330001,What is the smallest prime factor of 1821?,Step 1: Identify the properties of the number ...,[('Step 1: Identify the properties of the numb...,['Step 1: Identify the properties of the numbe...
2,req_330002,Billy weighs 9 pounds more than Brad. Brad wei...,Step 1: Determine Brad's weight based on Carl'...,"[(""Step 1: Determine Brad's weight based on Ca...","[""Step 1: Determine Brad's weight based on Car..."
3,req_330003,Simon and Peter have a big stamp collection. S...,Step 1: Calculate the total amount of money Si...,[('Step 1: Calculate the total amount of money...,['Step 1: Calculate the total amount of money ...
4,req_330004,A small zoo houses a variety of 68 wild animal...,Step 1: Determine the initial number of animal...,[('Step 1: Determine the initial number of ani...,['Step 1: Determine the initial number of anim...


In [15]:
df.iloc[0, 3]

'[(\'Step 1: Calculate the duration of jogging in minutes\', \'Since there are 60 minutes in an hour, half an hour is calculated as 60/2, which equals 30 minutes.\'), (\'Step 2: Determine the total calories burned during jogging\', \'Will burns 10 calories per minute, so for 30 minutes, the total calories burned is calculated as 10 * 30, which equals 300 calories.\'), ("Step 3: Calculate Will\'s net calorie intake after jogging", \'To find the net calorie intake, subtract the calories burned from the initial calorie intake: 900 (initial calories) - 300 (calories burned) = 600 calories.\'), (\'Conclusion:\', \'The answer is 600.\')]'

In [16]:
import os
print(os.getcwd())

/Users/nickklatsky/codingProjects/topicsFinal/Hierarchical_Reasoning_with_DLM/official_data_pipeline


In [17]:
df.describe()

# Rename HLR col as HLP
df = df.rename(columns={'HLR': 'HLP'})
df.head()


,custom_id,question,answer,LLR,HLP
0,req_330000,Will's breakfast supplied him 900 calories of ...,Step 1: Calculate the duration of jogging in m...,[('Step 1: Calculate the duration of jogging i...,['Step 1: Calculate the duration of jogging in...
1,req_330001,What is the smallest prime factor of 1821?,Step 1: Identify the properties of the number ...,[('Step 1: Identify the properties of the numb...,['Step 1: Identify the properties of the numbe...
2,req_330002,Billy weighs 9 pounds more than Brad. Brad wei...,Step 1: Determine Brad's weight based on Carl'...,"[(""Step 1: Determine Brad's weight based on Ca...","[""Step 1: Determine Brad's weight based on Car..."
3,req_330003,Simon and Peter have a big stamp collection. S...,Step 1: Calculate the total amount of money Si...,[('Step 1: Calculate the total amount of money...,['Step 1: Calculate the total amount of money ...
4,req_330004,A small zoo houses a variety of 68 wild animal...,Step 1: Determine the initial number of animal...,[('Step 1: Determine the initial number of ani...,['Step 1: Determine the initial number of anim...


In [18]:
import pandas as pd
import torch
from transformers import GPT2TokenizerFast
import ast

def clean_answer(text):
    text = text.replace('\n\n', ' ')
    text = text.replace('. Step', ' ; Step')
    text = text.rstrip('. ')
    return text

new_rows = []
for index, row in df.iterrows():
    test_q = row['question']
    test_a = row['answer']
    test_llr = row['LLR']
    test_hlr = row['HLP']

    test_q_HL = 'Question: ' + test_q + ' High Level Steps: '
    test_a_HL = ' ; '.join(ast.literal_eval(test_hlr)) + ' '

    test_a_LLR = clean_answer(test_a)

    test_full_HL = test_q_HL + test_a_HL
    test_full = 'Question: ' + test_q + ' Answer: ' + test_a_LLR

    new_rows.append({"high_level": test_full_HL, "low_level": test_full})




In [19]:
new_df = pd.DataFrame(new_rows)
new_df.head()

new_df.to_csv('../data/train_data_raw.csv')

# Output transformed dataset to data/train_data.csv